# Video Classification

This script consists all the steps for video classification.

1. extracting the images from the nii files.
2. converting the images of the nii file into the video.
3. converting the video into images again.
4. running extract_features.py to get the features from the vgg.
5. running train.py to train the deep neural model.



In [6]:
import os
import numpy as np
import nibabel as nib # to read nii files
import shutil # for file operations
import glob  # use to make file list from diectory
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import cv2 # opencv library
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from skimage import filters
from skimage import measure
from skimage.color import rgb2gray
from skimage.morphology import closing, square
import traceback
import sys

In [7]:
# Declaring the home variables that will be used throughout the script.

classes = ['Alzheimer',"MCI", 'Normal']
home_files_dir = '/home/ubuntu/Select_original_fmri/'

## Converting the nii files into images

In [ ]:
data = []
labels = []

#removing existing "images" (old data) folder from the directory '/home/ubuntu/Select_original_fmri/images'
try:
    shutil.rmtree(home_files_dir + 'images/')
except:
    print("No images folder in directory")
    pass

# making "images" (for new data) folder in the '/home/ubuntu/Select_original_fmri/images' directory
try:
    os.mkdir(home_files_dir + 'images/')
except:
    pass

# iterating /home/ubuntu/Select_original_fmri/niifiles/class_ (Alzheimer, MCI , Normal)
for class_ in classes:
    print ('working on ' + class_ + '...')
    
    for root, dir ,files in os.walk(os.path.join(home_files_dir+"niifiles/" , class_)): 
        
        # making class folder inside the '/home/ubuntu/Select_original_fmri/images' directory
        try:
            os.mkdir(home_files_dir + 'images/' + class_ + '/')
        except:
            pass
        
        # iterating through each nii file in each class in the '/home/ubuntu/Select_original_fmri/niifiles' directory
        for file_ in files:
            print 'working on ' + file_ + '...'
            
            store_path = home_files_dir + 'images/' + class_ + '/' + file_ + '/'
            
            try:
                os.mkdir(store_path)
            except:
                continue
            try:
                count = 0
                
                # extracting data from nii files
                x = nib.load(os.path.join(home_files_dir + 'niifiles/' , class_) + '/' + file_).get_data()
                
                # Including only 3 dimentions(length ,width,color) and excluding time dimention
                for i in xrange(x.shape[3]):
                    for j in xrange(x.shape[2]):
                            y = x[:, :, j, i]
                            img = Image.fromarray(y)
                            img = img.convert("RGB")
                            img = img.resize([224, 224])
                            
                            #Saving image in '/home/ubuntu/Select_original_fmri/image/class_/file'
                            img.save(store_path + 'i_' + str(count) + ".jpg")
                            count+=1
            except:
                print file_
    
    #os.system('rm -rf %s%s' %(home_files_dir2, class_))

# Converting Images into Video

In [11]:
videoclassification_dir="/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/"
try:
    # removing the completed directory in the videoclassification/data directory.
    shutil.rmtree(videoclassification_dir+"completed")
except:
    print('no completed directory in data directory')
    pass

try:
    # Making "videos" directory in vieoclassification/data
    os.mkdir(videoclassification_dir+"videos/")
except:
    shutil.rmtree(videoclassification_dir+"videos/")
    os.mkdir(videoclassification_dir+"videos/")

for class_ in classes:
    try:
        # making classes(Alzheimer,MCI, Normal) inside videos folder   
        os.mkdir(os.path.join(videoclassification_dir+"videos/" , class_))
    except:
        shutil.rmtree(os.path.join(videoclassification_dir+"videos" , class_))
        os.mkdir(os.path.join(videoclassification_dir+"videos/" , class_))
    
    # Extracting images from '/home/ubuntu/Select_original_fmri/images/class_'    
    for root, dirs ,files in os.walk(os.path.join(home_files_dir + "images/" , class_)):
        for dir_ in dirs:
            print 'working on ' + dir_ + "..." + class_
            # saving videos
            save_dir = videoclassification_dir+'videos/' + class_ + '/' + dir_ + '.avi'
            print os.system("avconv  -i " + home_files_dir + 'images/' + class_ + '/' + dir_ + '/i_%d.jpg' + " -c:v libx264 " + save_dir)
try:
    os.mkdir(videoclassification_dir+"completed")
except:
    pass

working on A25_018_S_5074.nii...Alzheimer
0
working on A27_018_S_4696.nii...Alzheimer
0
working on A78_019_S_4549.nii...Alzheimer
0
working on A15_100_S_5106.nii...Alzheimer
0
working on A41_130_S_4982.nii...Alzheimer
0
working on A48_130_S_4984.nii...Alzheimer
0
working on A16_053_S_5208.nii...Alzheimer
0
working on A1_136_S_4993.nii...Alzheimer
0
working on A57_130_S_5006.nii...Alzheimer
0
working on A20_019_S_5012.nii...Alzheimer
0
working on A19_019_S_5019.nii...Alzheimer
0
working on A93_130_S_4589.nii...Alzheimer
0
working on A33_006_S_4153.nii...Alzheimer
0
working on A58_006_S_4867.nii...Alzheimer
0
working on A29_010_S_5163.nii...Alzheimer
0
working on A34_002_S_5018.nii...Alzheimer
0
working on A90_053_S_5070.nii...Alzheimer
0
working on A26_018_S_4733.nii...Alzheimer
0
working on A98_130_S_4660.nii...Alzheimer
0
working on A53_006_S_4546.nii...Alzheimer
0
working on A63_013_S_5071.nii...Alzheimer
0
working on A74_019_S_4477.nii...Alzheimer
0
working on A55_130_S_4990.nii...A

In [12]:
path = "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/videos/"
dst = "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/"
files_list = [] 
test_list = []
train_list = []

count  = 0

try:
    os.mkdir(dst+"test/")
    os.mkdir(dst+"train/")
except:
    shutil.rmtree(dst+"test/")
    shutil.rmtree(dst+"train/")
    os.mkdir(dst+"test/")
    os.mkdir(dst+"train/")
    
for class_ in classes:
    print class_
    
    # dividing the data for training and testing purpose
    try:
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
    except:
        shutil.rmtree(dst+"train/"+class_)
        shutil.rmtree(dst+"test/"+class_)
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
        
    for root,dirs,files in os.walk(os.path.join(path,class_)):
        for f in files:
            files_list.append(f)
        test = int(len(files_list)*.3) # 30% test data
        train = int(round(len(files_list)*.7)) # 70% test data
        test_list += list(files_list[:test])
        train_list += list(files_list[test:train])
        for i in test_list:
            shutil.move(src=path+class_ +"/"+i, dst=dst+"test/"+class_+"/"+i)
        for i in train_list:
            shutil.move(src=path +  class_ +"/"+ i, dst=dst+"train/" +class_+"/"+i)
        files_list = []
        train_list = []
        test_list = []


Alzheimer
MCI
Normal


In [ ]:
% cd data

In [ ]:
!python 2_extract_files.py


In [ ]:
% cd ..

In [ ]:
!python extract_features.py

In [ ]:
!python train.py